In [1]:
import pandas as pd
import numpy as np

# Cleaning plan

## For weights just use the weight of the other competitor if possible ✅ 

## Locations just impute 'unkown' ✅

## For height try to see if they have a height in another fight and use that number ✅

## For winby just impute 'unkown'

## For age try to find the age in a previous fight and calculate age using the difference in dates

In [2]:
pd.options.display.max_columns = 30
pd.options.display.max_rows = 100

In [3]:
df = pd.read_csv('fights')

In [4]:
df.head()

,Unnamed: 0,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
0,0,0,0,38.0,193.0,Hounslow England,808,Amsterdam The Netherlands,Alistair Overeem,120.0,02/03/2014,646,4580,3,3,0,39.0,190.0,"Las Vegas, Nevada USA",377,"Las Vegas, Nevada USA",Frank Mir,119.0,DEC,blue,262.9,Cruiserweight
1,1,0,0,36.0,172.0,"Chicago, Illinois United States",1054,"Chicago, Illinois United States",Ricardo Lamas,65.0,02/03/2014,646,4589,5,5,0,32.0,170.0,Manaus Brazil,1052,Rio de Janeiro Brazil,Jose Aldo,65.0,DEC,red,143.0,Bantamweight
2,2,0,0,39.0,167.0,"Isla Vista , California USA",959,"Sacramento, California USA",Urijah Faber,61.0,02/03/2014,646,4590,1,5,0,31.0,167.0,Natal Brazil,1527,Rio de Janeiro Brazil,Renan Barao,61.0,KO/TKO,red,134.2,Flyweight
3,3,0,0,33.0,167.0,"San Diego, CA USA",1056,"San Diego, CA USA",Danny Martinez,56.0,02/03/2014,646,4605,3,3,0,37.0,160.0,"San Jose, California USA",1253,"Tucson, Arizona USA",Chris Cariaso,56.0,DEC,red,123.2,Strawwweight
4,4,0,0,36.0,185.0,Southampton England,2005,Southampton England,Tom Watson,84.0,02/03/2014,646,4631,3,3,0,37.0,182.0,"Englewood, NJ USA",464,"Brick, NJ USA",Nick Catone,84.0,DEC,red,184.8,Super Welterweight


In [5]:
[print(x) for x in df.B_HomeTown.values if type(x)==int]

[]

In [ ]:
df_nans = df[df.isna().any(axis=1)]
df_nans

## Clean heights ✅
 - Look for height record in different fight
  - seems like there are very few data to impute for Height
  - There are a lot of data for imputing in R_Height though but not for B_Height wich is strange
 - Use ID to search other fights

In [7]:
gimmie = df[df['R_ID'] == 2179]
# n = gimmie.iloc[0]['R_Height']
# pd.isna(n)
gimmie

,Unnamed: 0,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
307,307,0,0,31.0,175.0,Melbourne Australia,2295,Brisbane Australia,Brendan O'Reilly,77.0,08/25/2014,679,4963,3,3,1,NaN,175.0,NaN,2179,NaN,Lipeng Zhang,77.0,DEC,red,169.4,Super Lightweight
509,509,1,1,31.0,177.0,"Rockville Centre, New York United States",2313,"Islip, New York United States",Chris Wade,70.0,01/20/2015,698,5155,3,3,2,28.0,180.0,Inner Mongolia Province P.R. China,2179,Beijing China,Zhang Lipeng,70.0,DEC,blue,154.0,Featherweight
648,648,1,0,34.0,180.0,"Burns Lake, British Columbia Canada",2231,"Vancouver, British Columbia Canada",Kajan Johnson,70.0,05/18/2015,716,5329,3,3,3,28.0,180.0,Inner Mongolia Province P.R. China,2179,Beijing China,Zhang Lipeng,70.0,DEC,blue,154.0,Featherweight


In [8]:
# df['B_Height'] = df.groupby(['B_ID'])['B_Height'].ffill()

In [9]:
# Return all fights with that fighter where height isn't NaN
# Use that value to replace NaN height values for that fighter

def find_and_impute(data, impute, identifiers=[]):
    ''' 
    Takes in a dataframe, finds the nan values for "impute" row
    uses the id row to find non NaN value in the impute row
    '''
    d_copy = data.copy()
    
    print(d_copy[impute].isna().sum())
    for index, row in d_copy.iterrows():
        if pd.isna(row[impute]):
            ID = row[identifiers[0]]
            temp = df[df[identifiers[0]] == ID]
            temp2 = df[df[identifiers[1]] == ID]
            
            if temp.B_Height.mean():
                df.at[index, impute] = temp.B_Height.mean()
                
#             elif temp2.R_Height.mean():
#                 df.at[index, 'B_Height'] = temp2.R_Height.mean()
                
    print(df[impute].isna().sum())

In [10]:
find_and_impute(df, 'R_Height', ['B_ID', 'R_ID'])

0
0


In [11]:
find_and_impute(df, 'B_Height', ['R_ID', 'B_ID'])

3
3


In [12]:
# for index, row in df.iterrows():
#     if pd.isna(row.B_Height):
#         ID = row.B_ID
#         temp = df[df['B_ID'] == ID]
#         temp2 = df[df['R_ID'] == ID]
#         r_id.append(ID)
#         print(f'ID: {ID}')
#         print(temp.B_Height, temp2.R_Height)

In [13]:
# print(df.R_Height.isna().sum())
# for index, row in df.iterrows():
#     if pd.isna(row.R_Height):
#         ID = row.B_ID
#         temp = df[df['B_ID'] == ID]
#         temp2 = df[df['R_ID'] == ID]
#         if temp.B_Height.mean():
#             df.at[index, 'R_Height'] = temp.B_Height.mean()
#         elif temp2.R_Height.mean():
#             df.at[index, 'B_Height'] = temp2.R_Height.mean()
# print(df.R_Height.isna().sum())
#         print(f'ID: {ID}')
#         print(temp.B_Height, temp2.R_Height)

## impute for age 
 - Find the age in a fight and calculate age based on difference in time


In [14]:
# df1 = df[df.isna().any(axis=1)]
df1 = df[df..isna()]
df1

,Unnamed: 0,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
106,106,0,0,NaN,187.000000,NaN,2179,NaN,Lipeng Zhang,77.0,03/03/2014,655,4701,3,3,0,32.0,182.0,Shandong China,2180,Beijing China,Sai Wang,77.0,DEC,blue,169.4,Super Lightweight
180,180,0,0,NaN,183.600000,NaN,2246,NaN,Antonio Carlos Junior,84.0,06/02/2014,666,4879,3,3,0,39.0,185.0,"Joinvillve, Santa Catarina Brazil",2245,Rio de Janeiro Brazil,Vitor Miranda,84.0,DEC,blue,184.8,Super Welterweight
441,441,1,1,NaN,185.666667,NaN,2155,NaN,Aleksei Oleinik,120.0,10/07/2018,692,5040,1,3,2,32.0,187.0,"Sandpoint, Idaho USA",2105,"Arlington, Texas USA",Jared Rosholt,120.0,KO/TKO,blue,264.0,Cruiserweight
524,524,1,1,NaN,167.750000,NaN,2017,NaN,Cat Zingano,61.0,03/04/2015,700,5103,1,5,2,31.0,170.0,"Riverside, California USA",1828,"Venice, California USA",Ronda Rousey,61.0,SUB,red,134.2,Flyweight
647,647,1,0,NaN,167.000000,NaN,2298,NaN,Yao Zhikui,56.0,05/18/2015,716,5328,3,3,1,30.0,167.0,"Palm Springs, California USA",2303,"Huntington Beach, California USA",Nolan Ticman,56.0,DEC,blue,123.2,Strawwweight
1046,1046,4,0,NaN,187.400000,NaN,2246,NaN,Antonio Carlos Junior,84.0,03/21/2016,766,5966,3,3,4,40.0,182.0,"Melbourne, Victoria Australia",2324,"Melbourne, Victoria Australia",Daniel Kelly,84.0,KO/TKO,red,184.8,Super Welterweight
1374,1374,0,0,NaN,186.666667,NaN,2831,NaN,Jack Marshman,84.0,10/25/2017,795,6361,2,3,3,36.0,190.0,Stockholm Sweden,1938,Stockholm Sweden,Magnus Cedenblad,84.0,KO/TKO,blue,184.8,Super Welterweight
1713,1713,4,1,NaN,191.000000,NaN,2155,NaN,Aleksei Oleinik,115.0,07/10/2017,827,6728,2,3,7,36.0,200.0,"Honolulu, Hawaii United States",1515,"San Diego, California United States",Travis Browne,115.0,SUB,blue,253.0,Cruiserweight
1817,1817,0,0,NaN,167.000000,"Los Angeles, California United States",2972,"Las Vegas , Nevada United States",Donavon Frelow,61.0,08/09/2017,839,6812,3,3,0,NaN,170.0,"Pendelton, Oregon United States",2971,"Vancouver, Washington United States",Ricky Simon,61.0,DEC,red,134.2,Flyweight
2065,2065,9,4,NaN,183.000000,NaN,2246,NaN,Antonio Carlos Junior,84.0,04/16/2018,863,7139,1,3,9,37.0,182.0,"Lincolnville, Maine United States",435,"Sunbury, Pennsylvania United States",Tim Boetsch,84.0,SUB,blue,184.8,Super Welterweight


In [15]:
df[df.B_ID==2179]

,Unnamed: 0,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
106,106,0,0,NaN,187.0,NaN,2179,NaN,Lipeng Zhang,77.0,03/03/2014,655,4701,3,3,0,32.0,182.0,Shandong China,2180,Beijing China,Sai Wang,77.0,DEC,blue,169.4,Super Lightweight


In [16]:
type(df[df.R_ID==2179].Date[307])

str

In [118]:
cop = df.copy()
print(cop.shape)
cop = cop.dropna()
print(cop.shape)

(2317, 27)
(2233, 27)


In [119]:
2317-2233

84

In [96]:
# interate through data and find fights with missing age values
# Look for their age in a different fight and calculate the time difference of that nan
# Use the difference to calculate their aproximate age

def find_and_impute(data, impute, identifiers=[]):
    ''' 
    Takes in a dataframe, finds the nan values for "impute" row
    uses the id row to find non NaN value in the impute row
    '''
    d_copy = data.copy()
    
    print(d_copy[impute].isna().sum())
    for index, row in d_copy.iterrows():
        if pd.isna(row[impute]):
#             ID = row[f'{impute.split("_")[0]}_{identifiers[0].split("_")[1]}']
            ID = row[identifiers[0]]
            year = row.Date[-4:]
            ident_0 = d_copy[d_copy[identifiers[0]] == ID]
            ident_1 = d_copy[d_copy[identifiers[1]] == ID]
    
#             print('YEAR:', year)
            
            split = impute.split('_')
#             print(split)
            B_impute = f'B_{split[1]}'
            R_impute = f'R_{split[1]}'
            
            print(ID)
            
            if ~ident_0[B_impute].empty:
#                 loc = [x for x in ident_0[B_impute].values if type(x)==str]
#                 d_copy.at[index, impute] = ident_0[B_impute].values
                print(ident_0[B_impute].values)
#                 print(loc)
                
            if ~ident_1[R_impute].empty:
#                 loc = [x for x in ident_1[R_impute].values if type(x)==str]
#                 d_copy.at[index, impute] = ident_1[R_impute].values
                print(ident_1[R_impute].values)
#                 print(loc)
    print(d_copy[impute].isna().sum())

In [ ]:
find_and_impute(df, 'B_Age', ['R_ID', 'B_ID'])

In [ ]:
name = 'R_Age'
split = name.split('_')
df[f'R_{split[1]}']

In [47]:
R = 'R_HomeTown'
B = 'B_ID'
print(f'{R.split("_")[0]}_{B.split("_")[1]}')

R_ID


In [76]:
t_df = df[df.R_ID==2179]
type(t_df.R_HomeTown[307])

float

## impute for locations ✅
 - if location is NaN look for fighter ID in different fight and use that hometown
 - for fight location just impute unkown

In [99]:
find_and_impute(df, 'R_Location', ['B_ID', 'R_ID'])

24
2155
[nan nan nan]
[nan nan nan nan nan]
2179
[nan]
[nan 'Inner Mongolia Province P.R. China'
 'Inner Mongolia Province P.R. China']
2298
[nan]
[nan nan nan]
2017
[nan]
[nan nan nan nan]
2246
[nan nan nan]
[nan nan nan nan nan nan]
2246
[nan nan nan]
[nan nan nan nan nan nan]
2298
[nan]
[nan nan nan]
2246
[nan nan nan]
[nan nan nan nan nan nan]
2017
[nan]
[nan nan nan nan]
2155
[nan nan nan]
[nan nan nan nan nan]
2246
[nan nan nan]
[nan nan nan nan nan nan]
2298
[nan]
[nan nan nan]
2246
[nan nan nan]
[nan nan nan nan nan nan]
2155
[nan nan nan]
[nan nan nan nan nan]
2831
[nan]
[nan nan]
2246
[nan nan nan]
[nan nan nan nan nan nan]
2831
[nan]
[nan nan]
2284
[nan]
[nan]
2920
[]
[nan]
2914
[]
[nan]
2155
[nan nan nan]
[nan nan nan nan nan]
2017
[nan]
[nan nan nan nan]
2155
[nan nan nan]
[nan nan nan nan nan]
2017
[nan]
[nan nan nan nan]
24


In [ ]:
find_and_impute(df, 'B_HomeTown', ['B_ID', 'R_ID'])

In [101]:
find_and_impute(df, 'B_Location', ['Event_ID', 'Event_ID'])

12
2179
[]
[]
2246
[]
[]
2155
[]
[]
2017
[]
[]
2298
[]
[]
2727
[]
[]
2246
[]
[]
2831
[]
[]
2155
[]
[]
2246
[]
[]
3124
[]
[]
2155
[]
[]
12


In [24]:
# find_and_impute(df, 'B_Location', ['Event_ID', 'Event_ID'])

## Fixing weights ✅
 - fill na values with values of the other corner
 - calc the avg weight in lbs and add weightclass column

In [25]:
# Filling NaN weight values with value of the other corner

# df['B_Weight'] = df['B_Weight'].fillna(df['R_Weight'])
# df['R_Weight'] = df['R_Weight'].fillna(df['B_Weight'])

In [26]:
# df['avg_weight_lbs'] = (df.B_Weight + df.R_Weight)/2 * 2.2

In [27]:
# ranges = [-np.inf, 125.0, 135.0, 145.0, 155.0, 165.0, 170.0, 175.0, 185.0, 195.0, 205.0, 225.0, 265.0, np.inf]
# labels = ['Strawwweight','Flyweight', 'Bantamweight', 'Featherweight', 'Lightweight', 'Super Lightweight', 'Welterweight', 'Super Welterweight',
#           'Middleweight', 'Super Middleweight', 'Light Heavyweight', 'Cruiserweight', 'Heavyweight']
# weight_classes = df.weight_class = pd.cut(df.avg_weight_lbs, ranges, labels=labels)

In [28]:
# df['weight_class'] = pd.Series(weight_classes)

In [29]:
df.head(10)

,Unnamed: 0,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
0,0,0,0,38.0,193.0,Hounslow England,808,Amsterdam The Netherlands,Alistair Overeem,120.0,02/03/2014,646,4580,3,3,0,39.0,190.0,"Las Vegas, Nevada USA",377,"Las Vegas, Nevada USA",Frank Mir,119.0,DEC,blue,262.9,Cruiserweight
1,1,0,0,36.0,172.0,"Chicago, Illinois United States",1054,"Chicago, Illinois United States",Ricardo Lamas,65.0,02/03/2014,646,4589,5,5,0,32.0,170.0,Manaus Brazil,1052,Rio de Janeiro Brazil,Jose Aldo,65.0,DEC,red,143.0,Bantamweight
2,2,0,0,39.0,167.0,"Isla Vista , California USA",959,"Sacramento, California USA",Urijah Faber,61.0,02/03/2014,646,4590,1,5,0,31.0,167.0,Natal Brazil,1527,Rio de Janeiro Brazil,Renan Barao,61.0,KO/TKO,red,134.2,Flyweight
3,3,0,0,33.0,167.0,"San Diego, CA USA",1056,"San Diego, CA USA",Danny Martinez,56.0,02/03/2014,646,4605,3,3,0,37.0,160.0,"San Jose, California USA",1253,"Tucson, Arizona USA",Chris Cariaso,56.0,DEC,red,123.2,Strawwweight
4,4,0,0,36.0,185.0,Southampton England,2005,Southampton England,Tom Watson,84.0,02/03/2014,646,4631,3,3,0,37.0,182.0,"Englewood, NJ USA",464,"Brick, NJ USA",Nick Catone,84.0,DEC,red,184.8,Super Welterweight
5,5,0,0,35.0,180.0,Amazonas Brazil,2101,Rio de Janeiro Brazil,Alan Patrick,70.0,02/03/2014,646,4642,3,3,0,33.0,172.0,"Halifax, Nova Scotia Canada",1675,"Montreal, Quebec Canada",John Makdessi,70.0,DEC,blue,154.0,Featherweight
6,6,0,0,26.0,175.0,"Detroit, Michigan United States",2157,"Las Vegas, Nevada United States",Kevin Lee,70.0,02/03/2014,646,4649,3,3,0,31.0,175.0,"Wantagh, New York United States",1983,"Wantagh, New York United States",Al Iaquinta,70.0,DEC,red,154.0,Featherweight
7,7,0,0,33.0,175.0,Dagestan Russia,2158,"Coconut Creek, Florida United States",Rashid Magomedov,70.0,11/05/2016,646,4651,3,3,0,28.0,182.0,"Palos Heights, Illinois United States",2159,"Boston, Massachusetts United States",Tony Martin,70.0,DEC,blue,154.0,Featherweight
8,8,0,0,36.0,180.0,"Zinzeli, Astrakhan Oblast Russia",2160,"Zinzeli, Astrakhan Oblast Russia",Gasan Umalatov,77.0,02/03/2014,646,4652,3,3,0,31.0,190.0,"Brooklyn, New York United States",2047,"Denver, Colorado United States",Neil Magny,77.0,DEC,red,169.4,Super Lightweight
9,9,0,0,33.0,162.0,Dagestan Republic Russia,2084,Dagestan Republic Russia,Ali Bagautinov,56.0,02/03/2014,646,4658,3,3,0,28.0,160.0,"Paranagua, Parana Brazil",1966,"Paranagua, Parana Brazil",John Lineker,61.0,DEC,blue,128.7,Flyweight
